In [21]:
import requests
import os
from dotenv import load_dotenv
import pymongo
import time
from urllib.parse import urlparse


In [22]:
def get_current_timestamp_milliseconds():
  """
  Returns the current timestamp in milliseconds since the epoch.
  """
  return int(time.time() * 1000)

In [23]:
load_dotenv() 
LI_URI = os.environ['LI_URI']
LI_REST_URI = os.environ['LI_REST_URI']
LI_VERSION = os.environ['LI_VERSION']
LI_ACCESS_TOKEN = os.environ['LI_ACCESS_TOKEN']

In [24]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
db = db_client['db_li_page_posts']
tb_page_post = db['tb_page_posts']

In [25]:
def post_2_page(payload):
    #print(url)
    url = LI_REST_URI + 'posts'
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    try:
        detail = requests.post(url, json=payload, headers=headers)
        return detail   #LI does not return response detail
    except Exception as e:
        print(e)
        return {'error': e}

In [26]:
#get 1 latest post and repost to LI
def get_1_latest_pots():
    latest_post = tb_page_post.find_one({'shared': 0}, sort=[('lastModifiedAt', -1)])
    return latest_post
#test
#end

In [27]:
#get 1 random post to reshare if there were less than 5 posts in last 24 hours
def get_1_latest_post():
    timenow = get_current_timestamp_milliseconds()  #milliseconds
    last24hours = timenow - 24 * 60 * 60 * 1000
    todayPosts = tb_page_post.count_documents({'shared': 1, '$and': [ {'shared_time': {'$gt': last24hours }}, {'shared_time': {'$lt': timenow }} ] })
    # print(last24hours)
    print(todayPosts)
    if todayPosts < 5:
        #2. if today posted < 5 posts:
        #2.1 get 1 new post RANDOMLY, sorted by lastModifiedAt
        random_document = next(tb_page_post.aggregate([
            {"$match": {'shared': 0}},
            {"$sort": {"lastModifiedAt": -1}},
            #{"$sort": [("lastModifiedAt", -1)]},
            {"$sample": {"size": 1}}
        ]))
        return random_document
    #
    return None

In [28]:
#testing
the_post = tb_page_post.find_one({'id':'urn:li:share:7317871023838699520'})
the_post

{'_id': ObjectId('67ff3996b856f06050116ca7'),
 'id': 'urn:li:share:7317871023838699520',
 'lastModifiedAt': 1744716428119,
 'author': '25060884',
 'description': "5S Done Right = Real Results\nhttps://lnkd.in/gegCxzhc\n\n\xa05S isn’t about cleaning up — it’s about leveling up.\n\nWhen properly applied, 5S becomes a culture of clarity, flow, and safety.\n\nHere’s how each “S” adds real value:\n🧠 Sort – Remove clutter to focus on what matters\n📦 Set in Order – Organize for faster, mistake-free access\n✨ Shine – Clean to detect issues early\n📏 Standardize – Lock in consistency and reduce variation\n🔁 Sustain – Reinforce habits, accountability, and pride\n🛡️ Safety – Embed risk prevention into every step\n\nReal 5S = Real impact:\n🚀 Boosts productivity\n📉 Reduces waste\n🔧 Improves quality\n👷\u200d♂️ Increases safety\n💬 Lifts morale\n\nTo get results, measure what matters:\n🎯 Clear goals\n📊 Simple scoring\n🤝 Team involvement\n📱 Tech-enabled audits\n🧭 Built-in CI\n\nWant to take your 5S prog

In [29]:
#log the time when the post is shared
def update_shared_info(post_id, post_desc):
    tb_page_post.update_one({'id': post_id}, {'$set': {'shared': 1, 'shared_time': get_current_timestamp_milliseconds()}})
    print('Finished reshare to the page with description: ' + post_desc + ' ...')

In [30]:
#repost exactly, do not download (maximum 5 repost per day)
def auto_repost(random_document):
    #2 post it to page
    if random_document != None:
        payload = {
            "author": "urn:li:organization:" + os.environ['MY_PAGE_ID'],    #post to my page
            "commentary": '',   #the_post['description'], #duplicated content
            "visibility": "PUBLIC",
            "distribution": {
                "feedDistribution": "MAIN_FEED"
            },
            "lifecycleState": "PUBLISHED",
            "reshareContext": {
                "parent": random_document['id']
            }
        }
        #print(payload)
        result = post_2_page(payload)
        # print(result)
        if 'error' not in result:
            #2.3 Update to db: shared=1, shared_time=now
            update_shared_info(random_document['id'], random_document['description'][:30])

In [31]:
#download image into the folder
def download_img(image_url, img_name):
    folder_name = 'img' #in same place
    try:
        # Create the save folder if it doesn't exist
        os.makedirs(folder_name, exist_ok=True)
        # Get the filename from the URL
        file_path = os.path.join(folder_name, img_name + ".jpg")
        print(file_path)
        # Download the image
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes

        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Image downloaded successfully and saved to: {file_path}")
        return file_path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image from {image_url}: {e}")
    except OSError as e:
        print(f"Error creating or writing to file: {e}")
    return None #no file downloaded

In [32]:
#get upload link from LI
def get_upload_link():
    url = LI_URI + 'v2/assets?action=registerUpload'
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    payload = {
        "registerUploadRequest": {
            "recipes": [
                "urn:li:digitalmediaRecipe:feedshare-image"
            ],
            "owner": "urn:li:organization:" + os.environ['MY_PAGE_ID'],
            "serviceRelationships": [
                {
                    "relationshipType": "OWNER",
                    "identifier": "urn:li:userGeneratedContent"
                }
            ]
        }
    }
    try:
        detail = requests.post(url, json=payload, headers=headers)
        return detail.json()
    except Exception as e:
        print(e)
        return {'error': e}

In [33]:
#upload image to LI
def upload_img(file_path, upload_detail):
    uploadUrl = upload_detail['uploadMechanism']['com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest']['uploadUrl']
    asset = upload_detail['asset']
    #upload the image https://learn.microsoft.com/en-us/linkedin/consumer/integrations/self-serve/share-on-linkedin#upload-image-or-video-binary-file
    try:
        with open(file_path, 'rb') as file:
            files = {'file': (file.name, file, 'image/jpg')}  # You might need to adjust the Content-Type
            headers = {
                'Authorization': LI_ACCESS_TOKEN,
                'LinkedIn-Version': LI_VERSION
            }

            response = requests.post(uploadUrl, files=files, headers=headers)

            print("Status Code:", response.status_code)
            print("Headers:", response.headers)
            print("Response Body:", response.text)

            if response.status_code >= 200 and response.status_code < 300:
                print("File uploaded successfully!")
            else:
                print("File upload failed.")

    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error during upload: {e}")

    

In [34]:
#find the image link and reshare in LI page
def reshare_img(li_img_id):
    #find img details
    url = LI_REST_URI + 'images/' + li_img_id
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION
    }
    try:
        detail = requests.get(url, headers=headers)
        #print(detail.json())
        if 'downloadUrl' in detail.json():
            file_path = download_img(detail.json()['downloadUrl'], li_img_id.replace('urn:li:image:', ''))
            if file_path is not None:
                #download successfully, now upload to LI and share
                upload_link = get_upload_link()
                if 'error' not in upload_link and 'value' in upload_link:
                    upload_img(file_path, upload_link['value'])
                
    except Exception as e:
        print(e)

In [35]:
#download image and re-share into the page
def download_n_reshare_post():
    if the_post is None:
        return  #reach limit for last 24 hours, do not share anything
    #check type of the post
    if 'urn:li:image:' in the_post['media']:
        #download the image
        img_link = reshare_img(the_post['media'])
    else:
        #reshare the video, do not download
        auto_repost(the_post)

In [36]:
download_n_reshare_post()

img/D5622AQGVHelgCsM5zw.jpg
Image downloaded successfully and saved to: img/D5622AQGVHelgCsM5zw.jpg
Status Code: 201
Headers: {'Cache-Control': 'no-cache, no-store', 'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Set-Cookie': 'lang=v=2&lang=en-us; SameSite=None; Path=/; Domain=linkedin.com; Secure, bcookie="v=2&5bd78930-1031-434a-8b3c-d166ccfa72f4"; domain=.linkedin.com; Path=/; Secure; Expires=Thu, 16-Apr-2026 18:42:29 GMT; SameSite=None, bscookie="v=1&20250416184228c8d88966-4e50-4589-81e2-cb066e6cfcd7AQERGhgIqmfNEVfy37KHELNjxOzWxgB4"; domain=.www.linkedin.com; Path=/; Secure; Expires=Thu, 16-Apr-2026 18:42:29 GMT; HttpOnly; SameSite=None, lidc="b=TGST08:s=T:r=T:a=T:p=T:g=3019:u=1:x=1:i=1744828949:t=1744915349:v=2:sig=AQErRNin6fKC7707wY6NPk60tOxOclho"; Expires=Thu, 17 Apr 2025 18:42:29 GMT; domain=.linkedin.com; Path=/; SameSite=None; Secure', 'x-ambry-creation-time': 'Wed Apr 16 18:42:29 UTC 2025', 'access-control-allow-origin': 'https://www.linkedin.com', 'Strict

In [37]:
#db_client.close()